# uCov case study

## Import and functions

In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# 
def unique_uses(ref_suf, comp_suf, cols=["usageKind","usageType","usageFullyQualifiedName"]):
    unique_suf = ref_suf[cols].drop_duplicates().merge(comp_suf[cols].drop_duplicates(), how='left', indicator=True)
    return unique_suf[unique_suf['_merge'] == 'left_only'][cols]

## Loading data

In [4]:
sum = pd.read_csv("Spark-2.9.3.SUM.csv", sep="|")

# bug fix for initial ucov versions not adding "Method > VirtualInvocation" when a "Method > Invocation" is found
missing_data = sum[sum["usageKind"] == "SYMBOL_KIND_METHOD"][sum["usageType"] == "SYMBOL_USE_INVOCATION"]
missing_data['usageType'].replace({"SYMBOL_USE_INVOCATION": "SYMBOL_USE_VIRTUAL_INVOCATION"}, inplace=True)
sum = pd.concat([sum, missing_data], ignore_index=True)
sum.drop_duplicates(inplace=True)

sum['qualifiedUse'] = sum['usageKind'] + ' > ' + sum['usageType']
sum.head()

/tmp/ipykernel_5082/430220410.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missing_data = sum[sum["usageKind"] == "SYMBOL_KIND_METHOD"][sum["usageType"] == "SYMBOL_USE_INVOCATION"]


,projectId,projectType,projectLocation,usagePosition,usageEndPosition,usageFullyQualifiedName,usageKind,usageType,qualifiedUse
0,Spark-2.9.3,MAIN,I:/UCov/Repro/spark/Main,/src/main/java/spark/response.java(310:17),/src/main/java/spark/response.java(315:5),"spark.Response.removeCookie(java.lang.String,java.lang.String)",SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION
1,Spark-2.9.3,MAIN,I:/UCov/Repro/spark/Main,/src/main/java/spark/utils/sparkutils.java(34:32),/src/main/java/spark/utils/sparkutils.java(43:5),spark.utils.SparkUtils.convertRouteToList(java.lang.String),SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING
2,Spark-2.9.3,MAIN,I:/UCov/Repro/spark/Main,/src/main/java/spark/exceptionmapper.java(22:14),/src/main/java/spark/exceptionmapper.java(122:1),spark.ExceptionMapper,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE
3,Spark-2.9.3,MAIN,I:/UCov/Repro/spark/Main,/src/main/java/spark/queryparamsmap.java(261:21),/src/main/java/spark/queryparamsmap.java(263:5),spark.QueryParamsMap.values(),SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION
4,Spark-2.9.3,MAIN,I:/UCov/Repro/spark/Main,/src/main/java/spark/service.java(752:17),/src/main/java/spark/service.java(757:5),spark.Service.initExceptionHandler(java.util.function.Consumer),SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING


In [5]:
suf = pd.read_csv("Spark-2.9.3.SUF.csv", sep="|")

# bug fix for initial ucov versions not adding "Method > VirtualInvocation" when a "Method > Invocation" is found
missing_data = suf[suf["usageKind"] == "SYMBOL_KIND_METHOD"][suf["usageType"] == "SYMBOL_USE_INVOCATION"]
missing_data['usageType'].replace({"SYMBOL_USE_INVOCATION": "SYMBOL_USE_VIRTUAL_INVOCATION"}, inplace=True)
suf = pd.concat([suf, missing_data], ignore_index=True)
suf.drop_duplicates(inplace=True)

suf['qualifiedUse'] = suf['usageKind'] + ' > ' + suf['usageType']
suf['qualifiedSymbol'] = suf['usageFullyQualifiedName'] + ' > ' + suf['usageType']
suf.head()

/tmp/ipykernel_5082/1116322668.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missing_data = suf[suf["usageKind"] == "SYMBOL_KIND_METHOD"][suf["usageType"] == "SYMBOL_USE_INVOCATION"]


,projectId,projectType,projectLocation,usagePosition,usageEndPosition,usageFullyQualifiedName,usageKind,usageType,qualifiedUse,qualifiedSymbol
0,Spark-2.9.3,TEST,I:/UCov/Repro/spark/Tests,/src/test/java/spark/bodyavailabilitytest.java(32:9),/src/test/java/spark/bodyavailabilitytest.java(32:21),spark.Spark.stop(),SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,spark.Spark.stop() > SYMBOL_USE_STATIC_INVOCATION
1,Spark-2.9.3,TEST,I:/UCov/Repro/spark/Tests,/src/test/java/spark/bodyavailabilitytest.java(49:26),/src/test/java/spark/bodyavailabilitytest.java(52:9),"spark.Filter.handle(spark.Request,spark.Response)",SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,"spark.Filter.handle(spark.Request,spark.Response) > SYMBOL_USE_OVERRIDING"
2,Spark-2.9.3,TEST,I:/UCov/Repro/spark/Tests,/src/test/java/spark/bodyavailabilitytest.java(49:26),/src/test/java/spark/bodyavailabilitytest.java(52:9),spark.Filter,SYMBOL_KIND_INTERFACE,SYMBOL_USE_IMPLEMENTATION,SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,spark.Filter > SYMBOL_USE_IMPLEMENTATION
3,Spark-2.9.3,TEST,I:/UCov/Repro/spark/Tests,/src/test/java/spark/bodyavailabilitytest.java(49:9),/src/test/java/spark/bodyavailabilitytest.java(52:11),"spark.Spark.before(java.lang.String,spark.Filter)",SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.before(java.lang.String,spark.Filter) > SYMBOL_USE_STATIC_INVOCATION"
4,Spark-2.9.3,TEST,I:/UCov/Repro/spark/Tests,/src/test/java/spark/bodyavailabilitytest.java(50:51),/src/test/java/spark/bodyavailabilitytest.java(50:60),spark.Request.body(),SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,spark.Request.body() > SYMBOL_USE_INVOCATION


In [6]:
suf_test = suf[suf["projectType"] == "TEST"]
suf_sample = suf[suf["projectType"] == "SAMPLE"]
suf_client = suf[suf["projectType"] == "CLIENT"]

## Number of unique API interactions & unique API symbols

In [7]:
print(len(sum))
print(len(sum["usageFullyQualifiedName"].unique()))

1960
771


## Number of API symbols & interactions used in SUFs

In [8]:
suf_symbols = suf["usageFullyQualifiedName"].unique()
suf_uses_unique = suf["qualifiedSymbol"].unique()
suf_client_symbols = suf_client["usageFullyQualifiedName"].unique()
suf_client_uses_unique = suf_client["qualifiedSymbol"].unique()
suf_test_symbols = suf_test["usageFullyQualifiedName"].unique()
suf_test_uses_unique = suf_test["qualifiedSymbol"].unique()
suf_sample_symbols = suf_sample["usageFullyQualifiedName"].unique()
suf_sample_uses_unique = suf_sample["qualifiedSymbol"].unique()
print(f'All: {len(suf_symbols)} used, {len(suf)} uses, {len(suf_uses_unique)} unique.')
print(f'Clients: {len(suf_client_symbols)} used, {len(suf_client)} uses, {len(suf_client_uses_unique)} unique.')
print(f'Tests: {len(suf_test_symbols)} used, {len(suf_test)} uses, {len(suf_test_uses_unique)} unique.')
print(f'Samples: {len(suf_sample_symbols)} used, {len(suf_sample)} uses, {len(suf_sample_uses_unique)} unique.')

All: 301 used, 23882 uses, 476 unique.
Clients: 179 used, 20827 uses, 278 unique.
Tests: 248 used, 1605 uses, 400 unique.
Samples: 134 used, 1450 uses, 211 unique.


## SUM Analysis

### Prevalent API interactions in the SUM (~= profile)

In [9]:
display(sum["qualifiedUse"].value_counts(normalize=True).to_frame().style.bar())

,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,0.276531
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,0.274490
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,0.147959
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,0.111224
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,0.044898
SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_READ,0.034694
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,0.034694
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,0.030612
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,0.019898
SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_WRITE,0.010714


### Unused API symbols

In [10]:
sum_symbols_not_seen = unique_uses(sum, suf, cols=["usageFullyQualifiedName"])
sum_symbols_not_seen_client = unique_uses(sum, suf_client, cols=["usageFullyQualifiedName"])
sum_symbols_not_seen_sample = unique_uses(sum, suf_sample, cols=["usageFullyQualifiedName"])
sum_symbols_not_seen_test = unique_uses(sum, suf_test, cols=["usageFullyQualifiedName"])
sum_uses_not_seen = unique_uses(sum, suf, cols=["qualifiedUse"])

print(f'There are {len(sum_uses_not_seen)} kinds of uses in the SUM unseen in the SUFs.');
display(sum_uses_not_seen.head(n=15))

print("Kinds of uses that are the least covered")
display(unique_uses(sum, suf, cols=["usageFullyQualifiedName", "usageType"]).groupby(["usageType"]).size().sort_values(ascending=False).to_frame().style.bar())

print(f'There are {len(sum_symbols_not_seen)} unused API symbols for all SUFs.');
display(sum_symbols_not_seen.head(n=15))
print(f'There are {len(sum_symbols_not_seen_client)} unused API symbols in clients.');
display(sum_symbols_not_seen_client.head(n=15))
print(f'There are {len(sum_symbols_not_seen_sample)} unused API symbols in samples.');
display(sum_symbols_not_seen_sample.head(n=15))
print(f'There are {len(sum_symbols_not_seen_test)} unused API symbols in tests.');
display(sum_symbols_not_seen_test.head(n=15))

There are 2 kinds of uses in the SUM unseen in the SUFs.


,qualifiedUse
6,SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_READ
11,SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_WRITE


Kinds of uses that are the least covered


,0
usageType,
SYMBOL_USE_OVERRIDING,528
SYMBOL_USE_VIRTUAL_INVOCATION,382
SYMBOL_USE_INVOCATION,199
SYMBOL_USE_STATIC_INVOCATION,123
SYMBOL_USE_INSTANCE_FIELD_READ,68
SYMBOL_USE_INHERITANCE,56
SYMBOL_USE_TYPE_REFERENCE,56
SYMBOL_USE_INSTANTIATION,32
SYMBOL_USE_INSTANCE_FIELD_WRITE,21


There are 470 unused API symbols for all SUFs.


...

,usageFullyQualifiedName
0,"spark.Response.removeCookie(java.lang.String,java.lang.String)"
1,spark.utils.SparkUtils.convertRouteToList(java.lang.String)
2,spark.ExceptionMapper
3,spark.QueryParamsMap.values()
4,spark.Service.initExceptionHandler(java.util.function.Consumer)
5,spark.routematch.RouteMatch.getMatchUri()
7,spark.utils.ResourceUtils.isJarURL(java.net.URL)
8,"spark.embeddedserver.EmbeddedServerFactory.create(spark.route.Routes,spark.staticfiles.StaticFilesConfiguration,spark.ExceptionMapper,boolean)"
10,spark.RequestResponseFactory.create(javax.servlet.http.HttpServletResponse)
11,"spark.utils.urldecoding.UrlDecode.path(java.lang.String,int,int)"


There are 637 unused API symbols in samples.


,usageFullyQualifiedName
0,"spark.Response.removeCookie(java.lang.String,java.lang.String)"
1,spark.utils.SparkUtils.convertRouteToList(java.lang.String)
2,spark.ExceptionMapper
3,spark.QueryParamsMap.values()
4,spark.Service.initExceptionHandler(java.util.function.Consumer)
5,spark.routematch.RouteMatch.getMatchUri()
6,spark.Service.externalStaticFileLocation(java.lang.String)
7,spark.utils.ResourceUtils.isJarURL(java.net.URL)
8,"spark.embeddedserver.EmbeddedServerFactory.create(spark.route.Routes,spark.staticfiles.StaticFilesConfiguration,spark.ExceptionMapper,boolean)"
10,spark.RequestResponseFactory.create(javax.servlet.http.HttpServletResponse)


There are 523 unused API symbols in tests.


,usageFullyQualifiedName
3,spark.QueryParamsMap.values()
6,spark.Service.externalStaticFileLocation(java.lang.String)
7,spark.utils.ResourceUtils.isJarURL(java.net.URL)
10,spark.RequestResponseFactory.create(javax.servlet.http.HttpServletResponse)
11,"spark.utils.urldecoding.UrlDecode.path(java.lang.String,int,int)"
12,spark.Spark.awaitStop()
13,spark.resource.ClassPathResource.createRelative(java.lang.String)
16,spark.embeddedserver.EmbeddedServerFactory
17,spark.Spark.port(int)
18,spark.ResponseTransformerRouteImpl


### Unused API interactions in all SUFs

In [11]:
sum_not_seen = unique_uses(sum, suf)
print(len(sum_not_seen))
display(sum_not_seen.head(n=15))

1484


,usageKind,usageType,usageFullyQualifiedName
1,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.utils.SparkUtils.convertRouteToList(java.lang.String)
3,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,spark.QueryParamsMap.values()
4,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.Service.initExceptionHandler(java.util.function.Consumer)
5,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.routematch.RouteMatch.getMatchUri()
7,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.utils.ResourceUtils.isJarURL(java.net.URL)
10,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.RequestResponseFactory.create(javax.servlet.http.HttpServletResponse)
11,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,"spark.utils.urldecoding.UrlDecode.path(java.lang.String,int,int)"
13,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,spark.resource.ClassPathResource.createRelative(java.lang.String)
16,SYMBOL_KIND_INTERFACE,SYMBOL_USE_IMPLEMENTATION,spark.embeddedserver.EmbeddedServerFactory
17,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,spark.Spark.port(int)


### Unused API interactions in client

In [12]:
sum_not_seen = unique_uses(sum, suf_client)
print(len(sum_not_seen))
sum_not_seen.head(n=15)

1682


,usageKind,usageType,usageFullyQualifiedName
0,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,"spark.Response.removeCookie(java.lang.String,java.lang.String)"
1,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.utils.SparkUtils.convertRouteToList(java.lang.String)
2,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,spark.ExceptionMapper
3,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,spark.QueryParamsMap.values()
4,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.Service.initExceptionHandler(java.util.function.Consumer)
5,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.routematch.RouteMatch.getMatchUri()
7,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.utils.ResourceUtils.isJarURL(java.net.URL)
8,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,"spark.embeddedserver.EmbeddedServerFactory.create(spark.route.Routes,spark.staticfiles.StaticFilesConfiguration,spark.ExceptionMapper,boolean)"
10,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.RequestResponseFactory.create(javax.servlet.http.HttpServletResponse)
11,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,"spark.utils.urldecoding.UrlDecode.path(java.lang.String,int,int)"


### Most popular API symbols in all SUFs

In [13]:
suf.groupby(["usageFullyQualifiedName"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar()

,0
usageFullyQualifiedName,
spark.Request.queryParams(java.lang.String),2180
spark.Route,1900
"spark.Route.handle(spark.Request,spark.Response)",1764
spark.Request,1282
spark.Request.params(java.lang.String),1096
spark.Response,1023
"spark.Spark.get(java.lang.String,spark.Route)",1003
"spark.Response.header(java.lang.String,java.lang.String)",990
spark.Response.status(int),986


### Most popular API interactions

In [14]:
print("In all SUFs")
display(suf.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())
print("In clients")
display(suf_client.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())
print("In samples")
display(suf_sample.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())
print("In tests")
display(suf_test.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())

In all SUFs


In clients


In samples


In tests


### Least popular API symbols in all SUFs

  - Notice how some clients are invoking `*.internal.*` stuff!

In [15]:
suf.groupby(["usageFullyQualifiedName"]).size().sort_values(ascending=True).to_frame().head(n=15).style.bar()

,0
usageFullyQualifiedName,
"spark.utils.IOUtils.copy(java.io.InputStream,java.io.Writer)",1
"spark.Spark.before(java.lang.String,spark.Filter[])",1
"spark.Spark.webSocket(java.lang.String,java.lang.Object)",1
"spark.Spark.delete(java.lang.String,spark.TemplateViewRoute,spark.TemplateEngine)",1
spark.utils.IOUtils.toByteArray(java.io.InputStream),1
spark.embeddedserver.jetty.EmbeddedJettyFactory(),1
"spark.QueryParamsMap(java.lang.String,java.lang.String[])",1
spark.resource.UriPath.canonical(java.lang.String),1
spark.Spark.defaultResponseTransformer(spark.ResponseTransformer),1


### Least popular API interactions in all SUFs

In [16]:
suf.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=True).to_frame().head(n=15).style.bar()

### Most popular interaction kinds in SUFs

In [17]:
print("In all SUFs")
display(suf["qualifiedUse"].value_counts().to_frame().style.bar())
print("In clients")
display(suf_client["qualifiedUse"].value_counts().to_frame().style.bar())
print("In samples")
display(suf_sample["qualifiedUse"].value_counts().to_frame().style.bar())
print("In tests")
display(suf_test["qualifiedUse"].value_counts().to_frame().style.bar())

In all SUFs


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,6052
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,5807
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,3603
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,3553
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,2153
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,2120
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,286
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,286
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,20
SYMBOL_KIND_INTERFACE > SYMBOL_USE_EXTENSION,2


In clients


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,5305
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,5100
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,3274
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,3070
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,1831
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,1819
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,213
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,213
SYMBOL_KIND_INTERFACE > SYMBOL_USE_EXTENSION,1
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,1


In samples


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,336
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,315
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,199
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,197
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,152
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,136
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,50
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,50
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,14
SYMBOL_KIND_INTERFACE > SYMBOL_USE_EXTENSION,1


In tests


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,411
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,392
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,284
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,170
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,165
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,132
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,23
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,23
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,5


### Debug: is there any interaction found in the SUF that do not exist in the SUM?

In [18]:
suf_not_seen = unique_uses(suf, sum)
print(len(suf_not_seen))
display(suf_not_seen)

0


,usageKind,usageType,usageFullyQualifiedName


## SUF comparisons

    - Funny thing: there are samples using `*.internal.*` types!

In [19]:
only_client_not_in_test = unique_uses(suf_client, suf_test)
print(f'{len(only_client_not_in_test)} interactions done in clients, not in tests')
display(only_client_not_in_test.head(15))

only_client_not_in_sample = unique_uses(suf_client, suf_sample)
print(f'{len(only_client_not_in_sample)} interactions done in clients, not in samples')
display(only_client_not_in_sample.head(15))

only_sample_not_in_client = unique_uses(suf_sample, suf_client)
print(f'{len(only_sample_not_in_client)} interactions done in samples, not in clients')
display(only_sample_not_in_client.head(15))

only_sample_not_in_test = unique_uses(suf_sample, suf_test)
print(f'{len(only_sample_not_in_test)} interactions done in samples, not in tests')
display(only_sample_not_in_test.head(15))

# Union of interactions that we find in every client 'kind'?

71 interactions done in clients, not in tests


,usageKind,usageType,usageFullyQualifiedName
0,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.port(int)
8,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.options(java.lang.String,spark.Route)"
22,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,spark.Route
25,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,spark.ExceptionHandler
34,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,spark.servlet.SparkApplication.init()
38,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.port()
48,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.setPort(int)
50,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.post(java.lang.String,spark.TemplateViewRoute,spark.TemplateEngine)"
55,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.notFound(spark.Route)
63,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.ipAddress(java.lang.String)


83 interactions done in clients, not in samples


,usageKind,usageType,usageFullyQualifiedName
25,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,spark.ExceptionHandler
29,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.externalStaticFileLocation(java.lang.String)
31,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,spark.servlet.SparkApplication.init()
32,SYMBOL_KIND_INTERFACE,SYMBOL_USE_IMPLEMENTATION,spark.servlet.SparkApplication
33,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,spark.servlet.SparkApplication
34,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,spark.servlet.SparkApplication.init()
38,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.port()
48,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.setPort(int)
59,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.utils.IOUtils.toString(java.io.InputStream)
63,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.ipAddress(java.lang.String)


16 interactions done in samples, not in clients


,usageKind,usageType,usageFullyQualifiedName
29,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.put(java.lang.String,java.lang.String,spark.Route)"
59,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.modelAndView(java.lang.Object,java.lang.String)"
106,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,"spark.Redirect.post(java.lang.String,java.lang.String,spark.Redirect$Status)"
107,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,"spark.Redirect.any(java.lang.String,java.lang.String,spark.Redirect$Status)"
108,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.unmap(java.lang.String)
109,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.unmap(java.lang.String,java.lang.String)"
118,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,"spark.Service$StaticFiles.header(java.lang.String,java.lang.String)"
130,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,"spark.Response.cookie(java.lang.String,java.lang.String,int,boolean)"
137,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,spark.Response.type()
138,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,spark.Request.bodyAsBytes()


32 interactions done in samples, not in tests


,usageKind,usageType,usageFullyQualifiedName
1,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,spark.Route
7,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,spark.Request.params()
11,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.port(int)
28,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.delete(java.lang.String,spark.Route,spark.ResponseTransformer)"
29,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.put(java.lang.String,java.lang.String,spark.Route)"
30,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.post(java.lang.String,java.lang.String,spark.Route)"
44,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.options(java.lang.String,spark.Route)"
47,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.Spark.halt()
48,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"spark.Spark.post(java.lang.String,spark.TemplateViewRoute,spark.TemplateEngine)"
49,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,spark.utils.StringUtils.isEmpty(java.lang.Object)
